In [ ]:
import torch
import torchtext

from tensor import vocab, train_dataset, classes
from bow import train_epoch
from embedding import padify

vocab_size = len(vocab)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x, h = self.rnn(x)
        return self.fc(x.mean(dim=1))

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
network = RNNClassifier(vocab_size, embed_dim=64, hidden_dim=32, num_class=len(classes)).to(device)

In [ ]:
train_epoch(network, train_loader, learning_rate=0.001, epoch_size=10)